In [1]:
# Import the modules
import re 
import json
import glob
import csv 
import os 
from tqdm import tqdm

src_dir = "/media/rezwan/Study/Bengali_AI/hadith/hadith/"
dest_csv_dir = "/media/rezwan/Study/Bengali_AI/hadith/hadith/bukhari_muslim.csv"
hadith_books = ['bukhari', 'muslim']

## create list
_id = [] 
_src_book= []
_chp_no = []
_hadith_no = []
_narrator = []
_validity = []
_ref = []
_text_bn = []
_text_ar = []
_explanation = []
_extra_note = []
_preface = []

id_num = 0

for hadith in hadith_books:
    read_json_dir = os.path.join(src_dir, hadith)
    
    # which hadith book
    if hadith == 'bukhari':
        hadith_book_name = 'সহিহ বুখারী'
    elif hadith == 'muslim':
        hadith_book_name = 'সহিহ মুসলিম'

    for file in tqdm(sorted(glob.glob(os.path.join(read_json_dir, "*")))):
        # Opening JSON file
        f = open(file)
        # Convert JSON string to dictionary
        data = json.load(f)

        text_ar = data['data']['arabic']
        text_bn = data['data']['bangla']
        explanation = data['data']['explanation']
        extra_note = data['data']['extra-note']
        narrator = data['data']['narrator']
        validity = data['data']['validity']

        chapter_no = data['meta']['header']
        preface = data['meta']['preface']

        if preface != None:
            preface = " ".join(data['meta']['preface'])

        translator = text_bn
        # try:
        ref = None 
        if text_bn != None:
            bracket_ref = re.findall('\(.*?\)', text_bn) ##string within parenthesis
            if len(bracket_ref) == 0:
                ref = None 
            elif len(bracket_ref) > 1:
                ref = " ".join(bracket_ref[-1:-2])
        # except ValueError:

        # regex=u'(?<!\.)\n\n' #check \n\n 
        # _double_newline = re.findall(regex, text_bn)

        if text_bn != None:
            # "(?<!\.)\n\n"
            newline_pattern=u'(?<!\.)\n\n' #\n\n
            translator_ref=re.split(newline_pattern, text_bn)
            if len(translator_ref) > 1:
                ref = translator_ref[-1]
                translator = "".join(translator_ref[0:-1])

        _id.append(id_num)
        id_num += 1
        _src_book.append(hadith_book_name)
        _chp_no.append(chapter_no)

        jsno_file = file.split("/")[-1]
        hadith_no = jsno_file.split(".")[0]
        _hadith_no.append(int(hadith_no))

        _narrator.append(narrator)
        _validity.append(validity)
        _ref.append(ref)
        _text_bn.append(translator)
        _text_ar.append(text_ar)
        _explanation.append(explanation)
        _extra_note.append(extra_note)
        _preface.append(preface)
        
## Create dictionary
df_dict = {}

df_dict['id'] = _id
df_dict['source_book'] = _src_book
df_dict['chapter_no'] = _chp_no
df_dict['hadith_no'] = _hadith_no
df_dict['narrator'] = _narrator
df_dict['validity'] = _validity

df_dict['chain_idx'] = _ref
df_dict['text_bn'] = _text_bn
df_dict['text_ar'] = _text_ar
df_dict['explanation'] = _explanation
df_dict['extra_note'] = _extra_note
df_dict['preface'] = _preface

# create a csv file  
with open(dest_csv_dir, "w") as out_file:
    csv_file = csv.writer(out_file)
    csv_file.writerow(df_dict.keys())
    csv_file.writerows(zip(*df_dict.values()))

100%|██████████| 7500/7500 [07:09<00:00, 17.45it/s] 


In [ ]:
### json data format

'''
{
    "data": {
        "arabic": "",
        "bangla": "",
        "explanation": "",
        "extra-note": null,
        "narrator": "",
        "validity": ""
    },
    "meta": {
        "header": "",
        "preface": [
            "",
            ""
        ]
    }
}
'''